# LSXPS x FINK

In [1]:
import pandas as pd
import numpy as np

from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u

import swifttools.ukssdc.data.SXPS as uds

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')

### Download the LSXPS Catalog

In [3]:
sources_data = uds.getFullTable(table='sources',
                        returnData=True,
                        saveData=False,
                        silent=False,
                        verbose=True)

Uploading data to https://www.swift.ac.uk/API/main.php
Returned keys: dict_keys(['OK', 'URL', 'FILE', 'APIVersion'])
Checking returned data for required content.
Saving file `/tmp/tmpt7jxo5ge/LSXPS_Sources.csv`


ParserError: Error tokenizing data. C error: Expected 1 fields in line 13, saw 2
